In [3]:
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset

In [4]:
# Making tag list
entities = [
    "DAT", "EVT", "GPE", "LOC", "PER", "PRD", 
    "PRC", "CRD", "TIM", "NOR", 
    "ECONOMIC_INDICATOR", "ASSET_CLASS"
]

tag_list = ["O"]

for entity in entities:
    tag_list.append(f"B-{entity}")
    tag_list.append(f"I-{entity}")


In [7]:
with open('../../data/wordlist/tag.txt', 'w') as file:
    for tag in tag_list:
        file.write(tag + "\n")

# BERT

# 